In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import random

pi = 3.14159265359

maxval=1e9
minval=1e-9

2024-09-16 18:29:49.825839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#from dataprep import *
from OptimizedDataGenerator import OptimizedDataGenerator
from loss import *
from models import *

In [3]:
batch_size = 5000
val_batch_size = 5000
train_file_size = 50
val_file_size = 10
tfrecords_dir_train = "/uscms/home/jennetd/nobackup/smart-pixels/tfrecords/tfrecords_20t_train_50x12P5x100_bnorm1"
tfrecords_dir_val = "/uscms/home/jennetd/nobackup/smart-pixels/tfrecords/tfrecords_20t_val_50x12P5x100_bnorm1"

training_generator = OptimizedDataGenerator(
    data_directory_path = "/uscms/home/lagray/nobackup/smart-pixels/dataset8/unflipped-positive/data/",
    labels_directory_path = "/uscms/home/lagray/nobackup/smart-pixels/dataset8/unflipped-positive/labels/",
    is_directory_recursive = False,
    file_type = "parquet",
    data_format = "3D",
    batch_size = batch_size,
    file_count = train_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    scaling_list = [75.0, 18.75, 10.0, 1.22],
    input_shape = (20,13,21),
    transpose = (0,2,3,1),
    files_from_end=True,
    shuffle= True,
    
    tfrecords_dir = tfrecords_dir_train,
    use_time_stamps = -1, #-1
    max_workers = 1, # Don't make this too large (will use up all RAM)
    seed = 10,
    quantize = True # Quantization ON
)

validation_generator = OptimizedDataGenerator(
    data_directory_path = "/uscms/home/lagray/nobackup/smart-pixels/dataset8/unflipped-positive/data/",
    labels_directory_path = "/uscms/home/lagray/nobackup/smart-pixels/dataset8/unflipped-positive/labels/",
    is_directory_recursive = False,
    file_type = "parquet",
    data_format = "3D",
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    scaling_list = [75.0, 18.75, 10.0, 1.22],
    input_shape = (20,13,21),
    transpose = (0,2,3,1),
    files_from_end=True,
    shuffle= True,

    tfrecords_dir = tfrecords_dir_val,
    use_time_stamps = -1, #-1
    max_workers = 1, # Don't make this too large (will use up all RAM)
    seed = 10,
    quantize = True # Quantization ON
)

Directory /uscms/home/jennetd/nobackup/smart-pixels/tfrecords/tfrecords_20t_train_50x12P5x100_bnorm1 is removed...


Saving batches as TFRecords: 100%|██████████| 255/255 [08:12<00:00,  1.93s/it]


Directory /uscms/home/jennetd/nobackup/smart-pixels/tfrecords/tfrecords_20t_val_50x12P5x100_bnorm1 is removed...


Saving batches as TFRecords: 100%|██████████| 51/51 [01:42<00:00,  2.00s/it]


In [4]:
# compiles model
model=CreateModel((13,21,20),n_filters=5,pool_size=3)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 21, 20)]      0         
                                                                 
 q_separable_conv2d (QSepara  (None, 11, 19, 5)        285       
 bleConv2D)                                                      
                                                                 
 q_activation (QActivation)  (None, 11, 19, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 11, 19, 5)         30        
                                                                 
 q_activation_1 (QActivation  (None, 11, 19, 5)        0         
 )                                                               
                                                                 
 average_pooling2d (AverageP  (None, 3, 6, 5)          0     

In [5]:
model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss)

In [ ]:
# training
es = EarlyStopping(
    patience=50,
    restore_best_weights=True
)

base_dir = "./weights-50x12P5x100_bnorm1-checkpoints"
os.mkdir(base_dir)
checkpoint_filepath = base_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'
mcp = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=False,
)

class ScalePrintingCallback(keras.callbacks.Callback):    
    def on_epoch_end(self, epoch, logs=None):
        scale_layer = self.model.layers[-1]
        print(
            f"scaling layer ({epoch}):", 
            scale_layer.scale, 
            tf.math.softplus(scale_layer.scale)
        )

print_scale = ScalePrintingCallback()

history = model.fit(x=training_generator,
                    validation_data=validation_generator,
                    callbacks=[mcp],
                    epochs=600,
                    shuffle=False, # shuffling now occurs within the data-loader
                    verbose=1)

Epoch 1/600
118/255 [============>.................] - ETA: 2:23 - loss: 34266.5156